# Laboratorio de Introducción al Procesamiento de Lenguaje Natural

### Contexto

El objetivo de este laboratorio es introducirlos a la construcción de clasificadores, probando y comparando diferentes métodos.

### Entrega
Lo que deberán entregar es un archivo *.ipynb* con su solución, que incluya código, discusiones y conclusiones del trabajo. 

⚠️ Es importante que en el archivo a entregar estén **las salidas de cada celda ya ejecutadas** ⚠️. 

En caso de hacer el ejercicio 3, opcional, deberán entregar también un archivo .csv **correctamente formateado** con las predicciones de sus modelos.

El plazo máximo es el **21 de octubre a las 23:59 horas.**

### Plataforma sugerida
Sugerimos que utilicen la plataforma [Google colab](https://colab.research.google.com/), que permite trabajar colaborativamente con un *notebook* de python. Al finalizar pueden descargar ese *notebook* en un archivo .ipynb, incluyendo las salidas ya ejecutadas, con la opción ```File -> Download -> Download .ipynb```

### Instalación de bibliotecas
Antes de empezar, ejecuten esta celda para instalar las dependencias 👇

In [ ]:
!which python

/home/andres/fing/nsql/lab2/.venv/bin/python


In [ ]:
!python -m pip install sklearn
!python -m pip install gensim
!python -m pip install spacy
!python -m pip install nltk

# Ejercicio 1 - Primer contacto con el corpus

Lo primero a hacer es cargar el corpus. Hay muchas formas de hacerlo ([por ejemplo con Pandas](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)), pero la más sencilla es utilizando funcionalidades nativas de python. El resultado será una lista de n-uplas, donde cada una de ellas se correpondes a una fila del .csv (incluso el cabezal, la primera línea).

🧐**¿Qué tienen que hacer?**🤔 
Carguen a Colab los archivos necesarios del corpus usando el panel de la izquierda y luego ejecuten las siguientes celdas. Ajusten lo necesario para cargar todo el conjunto de *train* (`train_1.csv` y `train_2.csv`), el de dev y también sus anotaciones, que van a ser un subconjunto del archivo `train_1.csv`.

In [2]:
import csv

"""
  Completen con su código de carga de archivos "train" y "dev" acá
"""

with open('tweets_grupo_D_1.csv', newline='') as corpus_csv:
  reader = csv.reader(corpus_csv)
  next(reader) # Saltea el cabezal del archivo
  original_annotations = [x for x in reader]

with open('train_1.csv', newline='') as corpus_csv:
  reader = csv.reader(corpus_csv)
  next(reader) # Saltea el cabezal del archivo
  train_set = [x for x in reader]

with open('train_2.csv', newline='') as corpus_csv:
  reader = csv.reader(corpus_csv)
  next(reader) # Saltea el cabezal del archivo
  train_set = train_set + [x for x in reader] 

with open('dev.csv', newline='') as corpus_csv:
  reader = csv.reader(corpus_csv)
  next(reader) # Saltea el cabezal del archivo
  dev_set = [x for x in reader]

Imprimamos algún tweet aleatorio a ver si se cargó bien.

In [3]:
import random

# Elegir un tweet aleatorio e imprimirlo junto a su categoría
random_tweet = random.choice(dev_set)
print(f"El tweet es: {random_tweet[1]}")
print(f"y su categoría: {random_tweet[2]}")

El tweet es: Y bueno... #LaLiga tiene dueño 🎉🎊🎉🎊 #Barcelona #Barça #BarcaLevante Enhorabuena USER Gracias por tantas emociones, siempre 🔵🔴🔵🔴🔵🔴🔵🔴
y su categoría: joy


## Parte 1.1 - Composición de los conjuntos de entrenamiento y desarrollo

Para ver cómo esta compuesto el corpus, van a hacer una recorrida sobre todos los tweets en él y contar cuántos ejemplos hay de cada categoría. Examinen, discutan y comparen la cantidad de ejemplos en cada categoría, en *train* y en *dev* ¿hay más ejemplos de unas categorías que de otras? ¿tienen la misma proporción en *train* y *dev*?

🧐**¿Qué tienen que hacer?**🤔 Recorran los conjuntos, saquen conclusiones y escríbanlas en una celda de texto, a continuación.

In [4]:
import pprint

pp = pprint.PrettyPrinter(indent=4)

print(f"Train set length: {len(train_set)}")
print(f"Dev set length: {len(dev_set)}")

# Contar cuántos ejemplos hay de cada catergoría
# Train
train_set_categories = [tweet[2] for tweet in train_set]

category_names = set(train_set_categories)

train_set_categories_count = dict.fromkeys(category_names, 0)

for category in train_set_categories:
    train_set_categories_count[category] += 1

print("Train set tweet amount by category:")
pp.pprint(train_set_categories_count)

# Dev
dev_set_categories = [tweet[2] for tweet in dev_set]

category_names = set(dev_set_categories)

dev_set_categories_count = dict.fromkeys(category_names, 0)

for category in dev_set_categories:
    dev_set_categories_count[category] += 1

print("Dev set tweet amount by category:")
pp.pprint(dev_set_categories_count)

Train set length: 5723
Dev set length: 1250
Train set tweet amount by category:
{   'anger': 563,
    'disgust': 103,
    'fear': 63,
    'joy': 1230,
    'others': 2786,
    'sadness': 733,
    'surprise': 245}
Dev set tweet amount by category:
{   'anger': 150,
    'disgust': 27,
    'fear': 14,
    'joy': 259,
    'others': 617,
    'sadness': 131,
    'surprise': 52}


Aunque hay mas ejemplos de una que de otras, esto es esperable dado que el tamaño del conjunto de """**Dev**""" (no se como se llama en realidad) es más pequeño que el de train. Lo importante es que la distribución de los datos sea similar y como se puede observar en el resultado de la celda de código, esto se cumple.

In [5]:
print("Train set distribution")
pp.pprint({item[0]:round(item[1]/len(train_set),2) for item in train_set_categories_count.items()})
print("Dev set distribution")
pp.pprint({item[0]:round(item[1]/len(dev_set),2) for item in dev_set_categories_count.items()})

Train set distribution
{   'anger': 0.1,
    'disgust': 0.02,
    'fear': 0.01,
    'joy': 0.21,
    'others': 0.49,
    'sadness': 0.13,
    'surprise': 0.04}
Dev set distribution
{   'anger': 0.12,
    'disgust': 0.02,
    'fear': 0.01,
    'joy': 0.21,
    'others': 0.49,
    'sadness': 0.1,
    'surprise': 0.04}


## Parte 1.2 - Cálculo del acuerdo entre anotadores

A continuación queremos ver cuán de acuerdo estuvieron grupalmente con las anotaciones originales. Para eso deberán usar [esta función disponible en sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.cohen_kappa_score.html#sklearn.metrics.cohen_kappa_score).

🧐**¿Qué tienen que hacer?**🤔  Calculen el grado de acuerdo entre las antoaciones originales y las suyas como grupo. 

In [ ]:
from sklearn.metrics import cohen_kappa_score

original_annotations_categories = [annotation[2] for annotation in original_annotations]
train_set_categories = [annotation[2] for annotation in train_set[600:800]]

cohen_kappa_score(original_annotations_categories, train_set_categories)

0.401386263390044

TODO: Explicar un poco

# Ejercicio 2 - Experimentos con clasificadores

Ahora que cargaron y examinaron los datos, van a crear un primer clasificador para resolver el problema automáticamente. Como los clasificadores asumen que sus atributos son numéricos, hay que encontrar primero una forma numérica de representar los textos. En este ejercicio van a experimentar con varias formas de hacer eso.

En todas las partes podrán usar cualquiera de los clasificadores disponibles en el catálogo de modelos de [aprendizaje supervisado de sklearn](https://scikit-learn.org/stable/supervised_learning.html).  

## Parte 2.1 - Bag of Words

El primer experimento es utilizando Bag of Words (BoW) para representar los textos. Acá les dejamos un ejemplo, pero prueben con las diferentes configuraciones que admite [CountVectorizer de sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) y con los modelos que quieran del [catálogo de sklearn](https://scikit-learn.org/stable/supervised_learning.html). También pueden explorar diferentes formas de limpieza de los textos. 

Midan el aprendizaje sobre *dev* con la métrica [$F_1$, con la implementación de sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html). También pueden usar otras métricas adicionales; queda a su disposición.

🧐**¿Qué tienen que hacer?**🤔 Hagan varios experimentos con diferentes tipos de clasificadores y diferentes configuraciones de BoW para vectorizar. Midan el aprendizajen con $F_1$. Discutan, reflexionen y escriban las conclusiones en una celda de texto a continuación.

In [18]:
"""
  Completen con su código acá
"""

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
import warnings
warnings.filterwarnings('always')

# Posibles parametros a tocar: Strip accents, lowercase
bow_vectorizer = CountVectorizer()  # Vectorizador "bag of words"

classifiers = [
    ["multinomial", MultinomialNB()],
    ["decision_tree", DecisionTreeClassifier(random_state=0)],
    ["neighbours", KNeighborsClassifier(n_neighbors=10)],
    ["mlpc_classifier", MLPClassifier(random_state=0, max_iter=100)],
]

print("Bag of Words:")
print("Classifier       Accuracy    Precision    Recall    F-Score")
for clf in classifiers:
    training_features = bow_vectorizer.fit_transform(
        [x[1] for x in train_set]
    )  # Se vectorizan los tweets de train

    clf[1].fit(
        training_features, [x[2] for x in train_set]
    )  # Se entrena el clasificador usando los tweets vectorizados

    dev_features = bow_vectorizer.transform(
        [x[1] for x in dev_set]
    )  # Se vectorizan los tweets de dev
    prediction = clf[1].predict(
        dev_features
    )  # Se predicen las categorías de cada tweet (ya vectorizado en la línea anterior)

    accuracy = "{:.2f}".format(round(accuracy_score([x[2] for x in dev_set], prediction) * 100, 2))
    precision = "{:.2f}".format(round(precision_score([x[2] for x in dev_set], prediction, average="macro", zero_division=0) * 100, 2))
    recall = "{:.2f}".format(round(recall_score([x[2] for x in dev_set], prediction, average="macro") * 100, 2))
    fscore = "{:.2f}".format(round(f1_score([x[2] for x in dev_set], prediction, average="macro") * 100, 2))

    print(clf[0] + ' '*(17-len(clf[0])) + accuracy + ' '*7 + precision + ' '*8 + recall + ' '*5 + fscore)  # Se imprime la medida F


Bag of Words:
Classifier       Accuracy    Precision    Recall    F-Score
multinomial      61.44       52.35        30.00     30.30
decision_tree    54.96       39.90        37.40     38.41
neighbours       54.40       30.79        22.62     22.62
mlpc_classifier  60.32       50.76        41.52     44.39


Con los F-score obtenidos para los distintos clasificadores, entendemos que para los parámetros dados el clasificador MultiLayerPerception supera a los demás en clasificar el dev_set correctamente utilizando la vectorización Bag of Words. Analizando también los valores de Accuracy, Precision y Recall, entendemos que el MultinomialNB fue más acertivo en encontrar verdaderos positivos, pero resulta en un Recall menor al de MLP por obtener más falsos negativos.

## Parte 2.2 - TF-iDF

El segundo es utilizando TF-iDF (Term Frequency - inverse Document Frequency) para representar los textos.

🧐**¿Qué tienen que hacer?**🤔 Lo análogo a la parte anterior pero probando diferentes configuraciones con [TF-iDF de sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html). Comparen los resultados de $F_1$ con los obtenidos para Bag of Words y escriban las conclusiones en una celda de texto a continuación.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

# Posibles parametros a tocar: Strip accents, lowercase
tfid_vectorizer = TfidfVectorizer()
  # Vectorizador "Term Frequency - inverse Document Frequency"

classifiers = [
    ["multinomial", MultinomialNB()],
    ["decision_tree", DecisionTreeClassifier(random_state=0)],
    ["neighbours", KNeighborsClassifier(n_neighbors=10)],
    ["mlpc_classifier", MLPClassifier(random_state=0, max_iter=100)],
]

for clf in classifiers:
    training_features = tfid_vectorizer.fit_transform(
        [x[1] for x in train_set]
    )  # Se vectorizan los tweets de train
    clf[1].fit(
        training_features, [x[2] for x in train_set]
    )  # Se entrena el clasificador usando los tweets vectorizados

    dev_features = tfid_vectorizer.transform(
        [x[1] for x in dev_set]
    )  # Se vectorizan los tweets de dev
    prediction = clf[1].predict(
        dev_features
    )  # Se predicen las categorías de cada tweet (ya vectorizado en la línea anterior)

    print(
        f"F-Score {clf[0]}: "
        + str(
            round(
                f1_score([x[2] for x in dev_set], prediction, average="macro") * 100, 2
            )
        )
    )  # Se imprime la medida F


F-Score multinomial: 14.11
F-Score decision_tree: 38.69
F-Score neighbours: 38.99
F-Score mlpc_classifier: 44.15


## Parte 2.3 - Word embeddings

El tercer y último enfoque es utilizando **vectores de palabras estáticos** para representar los textos. Hay muchísimas colecciones de vectores de palabras, pero en esta ocasión vamos a usar unos entrenados por la Univerisdad de Chile. 

Una idea simple pero útil para representar un tweet puede ser hallar el centroide de los vectores relacionados a las palabras que aparecen en él, y luego comparar cuál es más similar a cuál. 

🧐**¿Qué tienen que hacer?**🤔 Lo análogo a las partes anteriores pero probando con una representación basada en *embeddings*. Comparen con $F_1$, saquen conclusiones y escríbanlas.


Les dejamos el siguiente código de ejemplo. Permite cargar los vectores, hallar el centroide de una lista de tokens y calcular las similitudes entre diferentes centroides.

In [ ]:
# Se descargan los vectores
!wget -q http://dcc.uchile.cl/~jperez/word-embeddings/fasttext-sbwc.100k.vec.gz
!gzip -d -q fasttext-sbwc.100k.vec.gz
!ls

dev.csv                 Letra_laboratorio.ipynb  train_2.csv
fasttext-sbwc.100k.vec  train_1.csv              tweets_grupo_D_1.csv


In [ ]:
from numpy.linalg import norm
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Se crea el objeto
vectors = KeyedVectors.load_word2vec_format('fasttext-sbwc.100k.vec')

# Unos ejemplos, ya tokenizados
example_1 = ["Qué", "tremendo", "día", "hace"]
example_2 = ["Qué", "lindo", "día", "hace"]
example_3 = ["Hace", "un", "precioso", "día"]
example_4 = ["Hoy", "está", "lindo", "el", "día"]
example_5 = ["Pah", "esta", "milanesa", "con", "mayonesa", "está", "buenísima"]
example_6 = ["Ya", "le", "dije", "que", "le", "vas", "a", "escribir"]

# Se calculan los centroides
centroid_example_1 = np.mean([vectors[word.lower()] for word in example_1], axis=0)
centroid_example_2 = np.mean([vectors[word.lower()] for word in example_2], axis=0)
centroid_example_3 = np.mean([vectors[word.lower()] for word in example_3], axis=0)
centroid_example_4 = np.mean([vectors[word.lower()] for word in example_4], axis=0)
centroid_example_5 = np.mean([vectors[word.lower()] for word in example_5], axis=0)
centroid_example_6 = np.mean([vectors[word.lower()] for word in example_6], axis=0)

# Se imprime la similitud entre los centroides del ejemplo 1 y el resto. 
print(cosine_similarity([centroid_example_1],[centroid_example_1]))
print(cosine_similarity([centroid_example_1],[centroid_example_2]))
print(cosine_similarity([centroid_example_1],[centroid_example_3]))
print(cosine_similarity([centroid_example_1],[centroid_example_4]))
print(cosine_similarity([centroid_example_1],[centroid_example_5]))
print(cosine_similarity([centroid_example_1],[centroid_example_6]))

[ 2.56507471e-02 -2.89550006e-01 -1.70209005e-01 -1.13342851e-01
  2.06646740e-01  4.34514955e-02 -2.02471748e-01 -3.34815197e-02
  4.63898294e-02  1.33737102e-01 -1.95077509e-02 -2.50672754e-02
  2.28695013e-02  1.34787261e-01  2.73027495e-02  8.70652497e-02
  1.17836520e-01  2.25424245e-01 -1.06735025e-02  3.15292478e-02
 -1.39987499e-01 -8.80450010e-02 -5.78614995e-02  1.26611248e-01
  2.04597488e-02 -1.91182494e-01  1.32874250e-01  4.12722528e-02
 -1.26742497e-01 -1.59242004e-01 -6.53272495e-03  2.06380002e-02
 -1.52943254e-01  6.02030009e-02 -5.00517488e-02 -3.97852510e-02
 -2.23585010e-01 -1.39164999e-01  1.64078012e-01  9.42077488e-02
  7.43749961e-02 -6.84305280e-02 -1.79119986e-02 -4.92537506e-02
 -1.61272511e-02  3.31189990e-01 -9.34137329e-02 -2.53442496e-01
 -9.29175019e-02  9.02502537e-02  2.39030495e-01  2.91985013e-02
  1.21482752e-01 -1.52302496e-02 -6.85224682e-03 -3.89675014e-02
 -6.83792531e-02 -2.81797498e-01  2.65489995e-01  4.67660017e-02
 -1.09462500e-01  1.75572

In [ ]:
"""
  Completen con su código acá
"""

tweets = [x[1] for x in train_set]

'\n  Completen con su código acá\n'

# Ejercicio 3 (opcional)

Esta última parte es **opcional**. Ahora que vieron cómo crear clasificadores, invitamos a que intenten construir el mejor clasificador posible utilizando estos enfoques o cualquier otro. Pueden probar lo que quieran, desde enfoques por reglas, utilizando POS-tagging, análisis sintáctico, análisis morfológico o listas de palabras, a modelos neuronales como BERT.

Si realizan esta parte opcional, tendrán que entregar en EVA las predicciones para un archivo de *test* que subiremos próximo a la entrega. Los grupos que obtengan las 3 mejores medidas al evaluar en el conjunto de test ganarán 5 puntos porcentuales que sumarán para la nota final del curso.

🧐**¿Qué tienen que hacer?**🤔 Construir el mejor clasificador posible y subir a EVA las predicciones para *test*.

In [ ]:
"""
  Completen con su código acá
"""

'\n  Completen con su código acá\n'